In [1]:
print('hello world')

hello world


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import sqlite3

# Get the European countries list
url = "https://en.wikipedia.org/wiki/List_of_European_countries_by_area"
response = requests.get(url)

# parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# find the table element of countries
table = soup.find('table', class_='wikitable sortable')
rows = table.find_all('tr')

# iterate through the rows and extract the country names from the 'state' column
countries_name = []
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 0:
        country = cells[1].text.replace("*", "").strip()
        countries_name.append(country)
countries_name.remove('Vatican City')
print(countries_name)

['Russia', 'Ukraine', 'France', 'Spain', 'Sweden', 'Norway', 'Germany', 'Finland', 'Poland', 'Italy', 'United Kingdom', 'Romania', 'Belarus', 'Kazakhstan', 'Greece', 'Bulgaria', 'Iceland', 'Hungary', 'Portugal', 'Serbia', 'Austria', 'Czechia', 'Ireland', 'Lithuania', 'Latvia', 'Croatia', 'Bosnia and Herzegovina', 'Slovakia', 'Estonia', 'Denmark', 'Switzerland', 'Netherlands', 'Moldova', 'Belgium', 'Albania', 'North Macedonia', 'Turkey', 'Slovenia', 'Montenegro', 'Azerbaijan', 'Georgia', 'Luxembourg', 'Andorra', 'Malta', 'Liechtenstein', 'San Marino', 'Monaco', 'Cyprus', 'Armenia']
